In [ ]:
import sys
import os
import torch

os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='1'

import numpy as np
import plotly.graph_objects as go

In [ ]:
def toDisplay(x, target_dim = 2):
    while(x.dim() > target_dim):
        x = x[0]
    return x.detach().cpu().numpy()

In [ ]:
import sys
sys.path.insert(1, '/home/exx/Documents/equivariant_pose_graph/python')
from equivariant_pose_graph.models.transformer_flow import ResidualFlow, ResidualFlow_V1
from equivariant_pose_graph.training.flow_equivariance_training_module import EquivarianceTrainingModule

# from ndf_robot.eval.test_trained_model_place import load_data
from pytorch3d.ops import sample_farthest_points
from pathlib import Path
import torch.nn.functional as F
from equivariant_pose_graph.utils.se3 import random_se3


In [ ]:
from equivariant_pose_graph.models.transformer_flow import ResidualFlow_DiffEmbTransformer
from equivariant_pose_graph.training.flow_equivariance_training_module_nocentering_eval_init_symmetry import EquivarianceTestingModule
checkpoint_file='/home/exx/media/DataDrive/singularity_chuerp/epg_results/residual_flow/residual_flow_occlusion_bottle_grasp_sym_breaking/2022-07-12_010420/residual_flow_occlusion_bottle_grasp_sym_breaking/3l22wmgi/checkpoints/epoch_431_global_step_54000.ckpt'
 
network = ResidualFlow_DiffEmbTransformer(
                    emb_nn='dgcnn', return_flow_component=False, center_feature=True,
                    inital_sampling_ratio=1)
model = EquivarianceTestingModule(
    network,
    lr=1e-4,
    image_log_period=100,
    weight_normalize='l1',
    softmax_temperature = 0.1,
    loop=1, 
    object_type = 'bottle'
)

model.cuda()
model.load_state_dict(torch.load(checkpoint_file)['state_dict'])




In [ ]:
def plot(points_action, points_anchor):
    colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]
    skip = 1
    points_action_dp = toDisplay(points_action)
    points_anchor_dp = toDisplay(points_anchor)
    go_data=[
        go.Scatter3d(x=points_action_dp[::skip,0], y=points_action_dp[::skip,1], z=points_action_dp[::skip,2], 
                     mode='markers', marker=dict(size=1, color=colors[0],
                     symbol='circle')),
        go.Scatter3d(x=points_anchor_dp[::skip,0], y=points_anchor_dp[::skip,1], z=points_anchor_dp[::skip,2], 
                     mode='markers', marker=dict(size=1, color=colors[1],
                     symbol='circle')),
    ]
    layout = go.Layout(
        scene=dict(
            aspectmode='data'
        )
    )

    fig = go.Figure(data=go_data, layout=layout)
    fig.show()

def plot_multi(plist):
    colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#e377c2',  # raspberry yogurt pink
    '#8c564b',  # chestnut brown
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]
    skip = 1
    go_data = []
    for i in range(len(plist)):
        p_dp = toDisplay(plist[i])
        plot = go.Scatter3d(x=p_dp[::skip,0], y=p_dp[::skip,1], z=p_dp[::skip,2], 
                     mode='markers', marker=dict(size=1, color=colors[i],
                     symbol='circle'))
        go_data.append(plot)
 
    layout = go.Layout(
        scene=dict(
            aspectmode='data'
        )
    )
 
    fig = go.Figure(data=go_data, layout=layout)
    fig.show()
def plot_color(plist, color_list):
    colors = [
    '#8c564b',  # chestnut brown
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]
    skip = 1
    go_data = []
    for i in range(len(plist)):
        if color_list[i]==None:
            p_dp = toDisplay(plist[i])
   
            plot = go.Scatter3d(x=p_dp[::skip,0], y=p_dp[::skip,1], z=p_dp[::skip,2], 
                         mode='markers', marker=dict(size=1, color=colors[i],
                         symbol='circle'))
            go_data.append(plot)
        else:
            p_dp = toDisplay(plist[i])
            color_list[i] = toDisplay(color_list[i])
            plot = go.Scatter3d(x=p_dp[::skip,0], y=p_dp[::skip,1], z=p_dp[::skip,2], 
                         mode='markers', marker=dict(size=1, color=color_list[i],
                         symbol='circle'))
            go_data.append(plot)
 
    layout = go.Layout(
        scene=dict(
            aspectmode='data'
        )
    )
 
    fig = go.Figure(data=go_data, layout=layout)
    fig.show()
    


def xyz2homo(xyz):
    """
    xyz:shape 1,num_points, 3
    """
    num_points = xyz.shape[1]
    homo = torch.cat([xyz.squeeze(0).detach().cpu(),torch.ones(num_points,1)],dim=-1)
    return homo
def transform(T,points):
    """
    points: num_points, 4
    """
    points = torch.permute(points,(-1,-2)) # 4,1000
    apply_here= torch.from_numpy(T).cuda()@points.cuda()
    apply_here = torch.permute(apply_here, (-1, -2))
    return apply_here[:,:3]

In [ ]:
def load_data(num_points, point_data, action_class, anchor_class):
    clouds = point_data['clouds'] 
    classes = point_data['classes']
    print("clouds.shape:{}".format(clouds.shape))
    print("classes.shape:{}".format(classes.shape))
    points_raw_np = clouds
    classes_raw_np = classes

    points_action_np = points_raw_np[classes_raw_np == action_class].copy()
#     points_action_mean_np = points_action_np.mean(axis=0)
#     points_action_np = points_action_np - points_action_mean_np
    
    points_anchor_np = points_raw_np[classes_raw_np == anchor_class].copy()
#     points_anchor_np = points_anchor_np - points_action_mean_np

    points_action = torch.from_numpy(points_action_np).float().unsqueeze(0)
    points_anchor = torch.from_numpy(points_anchor_np).float().unsqueeze(0)
    points_action, points_anchor = subsample(num_points,points_action, points_anchor)
    return points_action.cuda(), points_anchor.cuda()

def load_data2(num_points, point_data, action_class, anchor_class):
    clouds = point_data['clouds'] 
    classes = point_data['classes']
    pred_T_action_transformed = point_data["pred_T_action_transformed"]
    points_raw_np = clouds
    classes_raw_np = classes

    points_action_np = points_raw_np[classes_raw_np == action_class].copy()
    points_action_mean_np = points_action_np.mean(axis=0)
    pred_T_action_transformed = pred_T_action_transformed+ points_action_mean_np
    print("points_action_mean_np.shape:{}".format(points_action_mean_np.shape))
    
    points_anchor_np = points_raw_np[classes_raw_np == anchor_class].copy()
#     points_anchor_np = points_anchor_np - points_action_mean_np

    points_action = torch.from_numpy(points_action_np).float().unsqueeze(0)
    points_anchor = torch.from_numpy(points_anchor_np).float().unsqueeze(0)
    points_action, points_anchor = subsample(num_points,points_action, points_anchor)
    return torch.from_numpy(pred_T_action_transformed).cuda()

def subsample(num_points,points_action,points_anchor):
    if(points_action.shape[1] > num_points):
        points_action, _ = sample_farthest_points(points_action, 
            K=num_points, random_start_point=True)
    elif(points_action.shape[1] < num_points):
        raise NotImplementedError(f'Action point cloud is smaller than cloud size ({points_action.shape[1]} < {num_points})')

    if(points_anchor.shape[1] > num_points):
        points_anchor, _ = sample_farthest_points(points_anchor, 
            K=num_points, random_start_point=True)
    elif(points_anchor.shape[1] < num_points):
        raise NotImplementedError(f'Anchor point cloud is smaller than cloud size ({points_anchor.shape[1]} < {num_points})')
    
    return points_action, points_anchor

# def get_sym_label(num_points, point_data, action_class, anchor_class):
#     clouds = point_data['clouds'] 
#     classes = point_data['classes']
#     print("clouds.shape:{}".format(clouds.shape))
#     print("classes.shape:{}".format(classes.shape))
#     points_raw_np = clouds
#     classes_raw_np = classes
#     assert 0 in [action_class, anchor_class], "class 0 must be here somewhere as the manipulation object of interest"
#     if action_class == 0:
#         sym_breaking_class = action_class
#         center_class = anchor_class
#     elif anchor_class == 0:
#         sym_breaking_class = anchor_class
#         center_class = action_class

#     points_sym = points_raw_np[classes_raw_np == sym_breaking_class].copy()
#     points_nonsym = points_raw_np[classes_raw_np == center_class].copy()
#     non_sym_center = points_nonsym.mean(axis=0)
#     sym_center = points_sym.mean(axis=0)
#     sym2nonsym = torch.from_numpy(non_sym_center - sym_center) #(3,)
#     sym_vec = torch.from_numpy(points_sym - sym_center)
#     sym_cls = torch.sign(torch.matmul(sym_vec, sym2nonsym)).unsqueeze(-1) # num_points, 1
 
#     return sym_cls 



In [ ]:
def project_to_xy(vector):
    """
    vector: num_poins, 3
    """
    if len(vector.shape)>1:
        vector[:,-1] = 0
    elif len(vector.shape)==1:
        vector[-1] = 0
    return vector

def get_sym_label(action_cloud, anchor_cloud, action_class, anchor_class):

    assert 0 in [action_class, anchor_class], "class 0 must be here somewhere as the manipulation object of interest"
    if action_class == 0:
        sym_breaking_class = action_class
        center_class = anchor_class
        points_sym = action_cloud[0]
        points_nonsym = anchor_cloud[0]
    elif anchor_class == 0:
        sym_breaking_class = anchor_class
        center_class = action_class
        points_sym = anchor_cloud[0]
        points_nonsym = action_cloud[0]

    non_sym_center = points_nonsym.mean(axis=0)
    sym_center = points_sym.mean(axis=0)
    sym2nonsym = (non_sym_center - sym_center) 
    sym2nonsym = project_to_xy(sym2nonsym) # [a,b,0]
    orth_sym2nonsym = torch.zeros(sym2nonsym.shape).to(sym2nonsym.device)
    orth_sym2nonsym[0] = -orth_sym2nonsym[1]
    orth_sym2nonsym[1] = orth_sym2nonsym[0]
    
    sym_vec = points_sym- sym_center
    sym_vec = project_to_xy(sym_vec)
    
    sym_cls = torch.sign(torch.matmul(sym_vec, sym2nonsym)).unsqueeze(0) # num_points, 1
    
    return sym_cls 



In [ ]:
num_classes = 3
data_idx = 0
cloud_types = ['init', 'pre_grasp', 'post_grasp', 'teleport', 'post_place', 'final']
cloud_type = 'teleport'
# data_path = Path('/home/exx/Documents/ndf_robot/src/ndf_robot/test_grasp_place_demo_12_new') 
data_path = Path('/home/exx/Documents/ndf_robot/src/ndf_robot/test_bottle_upright_Seed2') 
point_data = np.load(data_path / f'{data_idx}_{cloud_type}_obj_points.npz', allow_pickle = True)

points_action, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= 2, anchor_class= 0)
sym_cls = get_sym_label(action_cloud = points_action, anchor_cloud = points_anchor, action_class= 2, anchor_class= 0) # num_points, 1
print((sym_cls>0).shape)
points_anchor_left = points_anchor[sym_cls>0]
points_anchor_right = points_anchor[sym_cls<=0]
plot_multi([points_anchor_left,points_anchor_right,points_action])

In [ ]:
def plot_multi_pca(pcd, axis, index_list = [0,1,2]):
    
    """
    pcd: point cloud data of shape (1,num_points, 3)
    axis: 3 principal component axis 
     
    """
    colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
    ]
    go_data = []
    skip=1
    p_dp = toDisplay(pcd)
    points = go.Scatter3d(x=p_dp[::skip,0], y=p_dp[::skip,1], z=p_dp[::skip,2], 
                     mode='markers', marker=dict(size=1, color=colors[-1],
                     symbol='circle'))
    go_data.append(points)
    points_mean = pcd[0].mean(axis=0).detach().cpu().numpy()
 
    for i in index_list:
        vector = go.Scatter3d( x = [points_mean[0],points_mean[0]+axis[i,0]],
                               y = [points_mean[1],points_mean[1]+axis[i,1]],
                               z = [points_mean[2],points_mean[2]+axis[i,2]],
                               marker = dict( size = 1,
                                              color = colors[i]),
                               line = dict(color = colors[i], width = 6))
        go_data.append(vector)
    
 
    layout = go.Layout(
        scene=dict(
            aspectmode='data'
        )
    )
 
    fig = go.Figure(data=go_data, layout=layout)
    fig.show()
def plot_multi_axis(pcd, axis):
    
    """
    pcd: point cloud data of shape (1,num_points, 3)
    axis: 3 principal component axis 
     
    """
    colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
    ]
    go_data = []
    skip=1
    p_dp = toDisplay(pcd)
    points = go.Scatter3d(x=p_dp[::skip,0], y=p_dp[::skip,1], z=p_dp[::skip,2], 
                     mode='markers', marker=dict(size=1, color=colors[-1],
                     symbol='circle'))
    go_data.append(points)
    points_mean = pcd[0].mean(axis=0).detach().cpu().numpy()
 

    vector = go.Scatter3d( x = [points_mean[0],points_mean[0]+axis[0]],
                           y = [points_mean[1],points_mean[1]+axis[1]],
                           z = [points_mean[2],points_mean[2]+axis[2]],
                           marker = dict( size = 1,
                                          color = colors[0]),
                           line = dict(color = colors[0], width = 6))
    go_data.append(vector)
    
 
    layout = go.Layout(
        scene=dict(
            aspectmode='data'
        )
    )
 
    fig = go.Figure(data=go_data, layout=layout)
    fig.show()
    
def plot_axis(plist, list_axis):
    
    """
    pcd: point cloud data of shape (1,num_points, 3)
    list_axis: [[axis_start1, axis_end1],[axis_start2, axis_end2],...]
     
    """
    colors = [
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf',   # blue-teal
    '#1f77b4',  # muted blue
    ]
    go_data = []
    skip=1
 
    for j in range(len(plist)):
        p_dp = toDisplay(plist[j])
        plot = go.Scatter3d(x=p_dp[::skip,0], y=p_dp[::skip,1], z=p_dp[::skip,2], 
                     mode='markers', marker=dict(size=1, color=colors[j],
                     symbol='circle'))
        go_data.append(plot)
    for i in range(len(list_axis)):
        axis_start = list_axis[i][0]
        axis_end = list_axis[i][1]
        vector = go.Scatter3d( x = [axis_start[0],axis_end[0]],
                               y = [axis_start[1],axis_end[1]],
                               z = [axis_start[2],axis_end[2]],
                               marker = dict( size = 1,
                                              color = colors[i+j]),
                               line = dict(color = colors[i+j], width = 6))
        go_data.append(vector)

 
    layout = go.Layout(
        scene=dict(
            aspectmode='data'
        )
    )
 
    fig = go.Figure(data=go_data, layout=layout)
    fig.show()

In [ ]:
import open3d as o3d
def to_np(tensor):
    return tensor.detach().cpu().numpy()

def to_torch(numpy, device_tensor = None):
    tensor = torch.from_numpy(numpy).double()
    if device_tensor != None:
        tensor = tensor.to(device_tensor.device)
    return tensor

def vec_norm(vec):
    """
    vec: (3) torch tensor
    """
    return vec/torch.norm(vec)

def dot_product(x,y):
    """
    x: (3) torch tensor
    y: (3) torch tensor
    """
    assert len(x.shape) ==1, "x should be of shape (3), but got (.., ..)"
    assert len(y.shape) ==1, "y should be of shape (3), but got (.., ..)"
    # norm vec
    return vec_norm(x).T@vec_norm(y)

def project_to_axis(vector,axis):
    """
    vector: (3)
    axis: 3, can be not unit-normed
    """
    # make axis unit normed
    axis = vec_norm(axis)
    vector = vector.double()
    projected_vec = (vector@axis)*axis
    return projected_vec

def get_sym_label_pca(action_cloud, anchor_cloud, action_class, anchor_class):
    assert 0 in [action_class, anchor_class], "class 0 must be here somewhere as the manipulation object of interest"
    if action_class == 0:
        sym_breaking_class = action_class
        center_class = anchor_class
        points_sym = action_cloud[0]
        points_nonsym = anchor_cloud[0]
    elif anchor_class == 0:
        sym_breaking_class = anchor_class
        center_class = action_class
        points_sym = anchor_cloud[0]
        points_nonsym = action_cloud[0]

    non_sym_center = points_nonsym.mean(axis=0)
    points_sym_np = to_np(points_sym)
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_sym_np)
    pcd_mean, pcd_cov = pcd.compute_mean_and_covariance()
    evals, evecs = np.linalg.eig(pcd_cov) 
    evecs = np.transpose(evecs)
    major_axis = evecs[0]
    
    major_axis = to_torch(major_axis, device_tensor = non_sym_center)
    pcd_mean = to_torch(pcd_mean, device_tensor = non_sym_center)  
    projected_point = project_to_axis(vector = non_sym_center-pcd_mean, axis = major_axis) + pcd_mean 
     
    print("dot product should be 1")
    self_dot = dot_product(projected_point-pcd_mean, major_axis)
    print(self_dot.item()) 
    
    air_point = non_sym_center+(pcd_mean-projected_point)
    sym_vec = points_sym - pcd_mean
    sym2nonsym = air_point-pcd_mean
    vec = to_np(projected_point - pcd_mean)
    axes = [[to_np(pcd_mean), to_np(projected_point)],
            [to_np(projected_point), to_np(non_sym_center)],
            [to_np(pcd_mean), to_np(major_axis+pcd_mean)],
            [to_np(pcd_mean), to_np(air_point)]]
    # plot_axis([anchor_cloud, action_cloud], axes) 
#     print(torch.matmul(sym_vec, sym2nonsym))
    
    projected_point_0 =  project_to_axis(vector = points_sym[0]-pcd_mean, axis = major_axis) + pcd_mean 
    
    sym_cls = torch.sign(torch.matmul(sym_vec, sym2nonsym)).unsqueeze(0) # num_points, 1
    return sym_cls 

def get_sym_label_pca_cts(action_cloud, anchor_cloud, action_class, anchor_class):
    assert 0 in [action_class, anchor_class], "class 0 must be here somewhere as the manipulation object of interest"
    if action_class == 0:
        sym_breaking_class = action_class
        center_class = anchor_class
        points_sym = action_cloud[0]
        points_nonsym = anchor_cloud[0]
    elif anchor_class == 0:
        sym_breaking_class = anchor_class
        center_class = action_class
        points_sym = anchor_cloud[0]
        points_nonsym = action_cloud[0]

    non_sym_center = points_nonsym.mean(axis=0)
    points_sym_np = to_np(points_sym)
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_sym_np)
    pcd_mean, pcd_cov = pcd.compute_mean_and_covariance()
    evals, evecs = np.linalg.eig(pcd_cov) 
    evecs = np.transpose(evecs)
    major_axis = evecs[0]
    
    major_axis = to_torch(major_axis, device_tensor = non_sym_center)
    pcd_mean = to_torch(pcd_mean, device_tensor = non_sym_center)  
    projected_point = project_to_axis(vector = non_sym_center-pcd_mean, axis = major_axis) + pcd_mean 
     
    print("dot product should be 1")
    self_dot = dot_product(projected_point-pcd_mean, major_axis)
    print(self_dot.item()) 
    
    air_point = non_sym_center+(pcd_mean-projected_point)
    sym_vec = points_sym - pcd_mean
    sym2nonsym = air_point-pcd_mean
    vec = to_np(projected_point - pcd_mean)
    axes = [[to_np(pcd_mean), to_np(projected_point)],
            [to_np(projected_point), to_np(non_sym_center)],
            [to_np(pcd_mean), to_np(major_axis+pcd_mean)],
            [to_np(pcd_mean), to_np(air_point)]]
    # plot_axis([anchor_cloud, action_cloud], axes) 
#     print(torch.matmul(sym_vec, sym2nonsym))
    mcenter_point = sym_vec[0]
#     # radial distance from major axis
#     projected_point_0 = project_to_axis(vector = mcenter_point, axis = major_axis) 
#     dist = torch.norm(projected_point_0 - mcenter_point) # radial distance from major axis

#     projected_point_sym2nonsym = project_to_axis(vector = mcenter_point, axis = sym2nonsym) 
#     dist_projected_point_sym2nonsym = torch.norm(projected_point_sym2nonsym) 
    axes0 = [
            # [to_np(projected_point_0+pcd_mean), to_np(mcenter_point+pcd_mean)],
             [to_np(projected_point_sym2nonsym+pcd_mean), to_np(pcd_mean)],
             [to_np(sym2nonsym+pcd_mean),to_np(pcd_mean)],
             [to_np(mcenter_point+pcd_mean),to_np(pcd_mean)],
             [to_np(major_axis+pcd_mean),to_np(pcd_mean)]]
    plot_axis([anchor_cloud, action_cloud],axes0)
    dotproduct = torch.matmul(mcenter_point,sym2nonsym)/torch.norm(sym2nonsym)
 
    project_to_axis(vector = points_sym[0]-pcd_mean, axis = major_axis)
    sym_cls = torch.sign(torch.matmul(sym_vec, sym2nonsym)).unsqueeze(0) # num_points, 1

    return sym_cls 

In [ ]:
def get_sym_label_pca_bowl_grasp(action_cloud, anchor_cloud, action_class, anchor_class, object_type = 'bowl'):
    assert 0 in [action_class, anchor_class], "class 0 must be here somewhere as the manipulation object of interest"
    if action_class == 0:
        sym_breaking_class = action_class
        center_class = anchor_class
        points_sym = action_cloud[0]
        points_nonsym = anchor_cloud[0]
    elif anchor_class == 0:
        sym_breaking_class = anchor_class
        center_class = action_class
        points_sym = anchor_cloud[0]
        points_nonsym = action_cloud[0]

    non_sym_center = points_nonsym.mean(axis=0)
    points_sym_np = to_np(points_sym)
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_sym_np)
    pcd_mean, pcd_cov = pcd.compute_mean_and_covariance()
    evals, evecs = np.linalg.eig(pcd_cov) 
    evecs = np.transpose(evecs)
    major_axis = evecs[-1]
    if object_type == 'bottle':
        major_axis = evecs[0]
    
    plot_multi_pca(points_sym.unsqueeze(0), axis = evecs)
    
    major_axis = to_torch(major_axis, device_tensor = non_sym_center)
    pcd_mean = to_torch(pcd_mean, device_tensor = non_sym_center)  
    projected_point = project_to_axis(vector = non_sym_center-pcd_mean, axis = major_axis) + pcd_mean 
     
    print("dot product should be 1")
    self_dot = dot_product(projected_point-pcd_mean, major_axis)
    print(self_dot.item()) 
    
    air_point = non_sym_center+(pcd_mean-projected_point)
    sym_vec = points_sym - pcd_mean
    sym2nonsym = air_point-pcd_mean
    if object_type == 'bottle':
        sym2nonsym = torch.cross(sym2nonsym, major_axis)
    vec = to_np(projected_point - pcd_mean)
    axes = [[to_np(pcd_mean), to_np(projected_point)],
            [to_np(projected_point), to_np(non_sym_center)],
            [to_np(pcd_mean), to_np(major_axis+pcd_mean)],
            [to_np(pcd_mean), to_np(air_point)]]
    # plot_axis([anchor_cloud, action_cloud], axes) 
#     print(torch.matmul(sym_vec, sym2nonsym))
    mcenter_point = sym_vec[0]
    # radial distance from major axis
    projected_point_0 = project_to_axis(vector = mcenter_point, axis = major_axis) 
    dist = torch.norm(projected_point_0 - mcenter_point) # radial distance from major axis

    projected_point_sym2nonsym = project_to_axis(vector = mcenter_point, axis = sym2nonsym) 
    dist_projected_point_sym2nonsym = torch.norm(projected_point_sym2nonsym) 
    axes0 = [
             [to_np(non_sym_center), to_np(pcd_mean)],
             [to_np(projected_point_sym2nonsym+pcd_mean), to_np(pcd_mean)],
             [to_np(sym2nonsym+pcd_mean),to_np(pcd_mean)],
             [to_np(mcenter_point+pcd_mean),to_np(pcd_mean)],
             [to_np(major_axis+pcd_mean),to_np(pcd_mean)]]
    plot_axis([anchor_cloud, action_cloud],axes0)
    dotproduct = torch.matmul(mcenter_point,sym2nonsym)/torch.norm(sym2nonsym)
 
    project_to_axis(vector = points_sym[0]-pcd_mean, axis = major_axis)
    sym_cls = torch.sign(torch.matmul(sym_vec, sym2nonsym)) # 1, num_points 
    cts_cls =  torch.matmul(sym_vec,sym2nonsym)/torch.norm(sym2nonsym) 
    cts_cls_norm = cts_cls/torch.max(torch.abs(cts_cls))
    print("cts_cls_norm", torch.max(cts_cls_norm), torch.min(cts_cls_norm), cts_cls_norm.shape)
    print("cts_cls.shape",cts_cls.shape,points_sym.shape)
    # coloring cts_cls for vis (+: red, -: blue)
    color = (cts_cls/torch.abs(torch.max(cts_cls))*255)
    color_cts = torch.zeros(points_sym.shape).to(points_sym.device).double()
 
    color_cts[cts_cls>=0,0] = color[cts_cls>=0]
    color_cts[cts_cls<0,2] = torch.abs(color[cts_cls<0])
    color_cts[cts_cls>=0]
    
    # another color scheme (+: red, -: green, 0: yellow )
    # https://stackoverflow.com/questions/6394304/algorithm-how-do-i-fade-from-red-to-green-via-yellow-using-rgb-values
    color = (cts_cls/torch.abs(torch.max(cts_cls))*(255/2))+(255/2)
    color_cts = torch.zeros(points_sym.shape).to(points_sym.device).double()
 
    color_cts[:,0] = torch.minimum(torch.ones(color.shape).to(color.device)*255,color*2)
    color_cts[:,1] = torch.minimum(torch.ones(color.shape).to(color.device)*255,(255-color)*2)
    plot_color([points_sym,points_nonsym],[color_cts,None])
    return sym_cls.unsqueeze(0), cts_cls.unsqueeze(0)


In [ ]:
def get_sym_label_pca_bowl_place(action_cloud, anchor_cloud, action_class, anchor_class, object_type = 'bowl'):
    assert 0 in [action_class, anchor_class], "class 0 must be here somewhere as the manipulation object of interest"
    if action_class == 0:
        sym_breaking_class = action_class
        center_class = anchor_class
        points_sym = action_cloud[0]
        points_nonsym = anchor_cloud[0]
    elif anchor_class == 0:
        sym_breaking_class = anchor_class
        center_class = action_class
        points_sym = anchor_cloud[0]
        points_nonsym = action_cloud[0]

    non_sym_center = points_nonsym.mean(axis=0)
    points_sym_np = to_np(points_sym)
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_sym_np)
    pcd_mean, pcd_cov = pcd.compute_mean_and_covariance()
    evals, evecs = np.linalg.eig(pcd_cov) 
    evecs = np.transpose(evecs)
    major_axis = evecs[0]
    major_axis = to_torch(major_axis, device_tensor = non_sym_center)
    radial_axis = torch.cross(major_axis,to_torch(evecs[1],device_tensor = major_axis))
    
 
    
    plot_multi_pca(points_sym.unsqueeze(0), axis = evecs)
    
    points_nonsym_np = to_np(points_nonsym)
    pcd_nonsym = o3d.geometry.PointCloud()
    pcd_nonsym.points = o3d.utility.Vector3dVector(points_nonsym_np)
    pcd_nonsym_mean, pcd_nonsym_cov = pcd_nonsym.compute_mean_and_covariance()
    nonsym_evals, nonsym_evecs = np.linalg.eig(pcd_nonsym_cov) 
    nonsym_evecs = np.transpose(nonsym_evecs)
    nonsym_major_axis = nonsym_evecs[0]
    nonsym_major_axis = to_torch(nonsym_major_axis, device_tensor = non_sym_center)
    
    pcd_mean = to_torch(pcd_mean, device_tensor = non_sym_center)  
    sym2nonsym = pcd_mean-non_sym_center
 
    symmetry_plane_normal = nonsym_major_axis 
    sym_vec = points_sym-pcd_mean
    
    # make sure the z-axis always point towards the opening of the bowl
     
    if object_type == 'bowl':
        radial_dist = torch.matmul(sym_vec,radial_axis)/torch.norm(radial_axis)
        max_radial_idx = torch.argmax(radial_dist)
        max_radial_vec = sym_vec[max_radial_idx]
        sign = torch.sign(torch.matmul(max_radial_vec,major_axis))
        print('sign',sign)


    if object_type == 'bottle':
        vertial_dist = torch.matmul(sym_vec,major_axis)/torch.norm(major_axis)
        max_radial_idx = torch.argmax(vertial_dist)
        max_radial_vec = sym_vec[max_radial_idx]
        min_radial_idx = torch.argmin(vertial_dist)
        min_radial_vec = sym_vec[min_radial_idx]
        radial_dist = torch.matmul(sym_vec,radial_axis)/torch.norm(radial_axis)
        if torch.abs(radial_dist[max_radial_idx]) < torch.abs(radial_dist[min_radial_idx]):
            sign = 1
        else:
            sign = -1
    if sign <0:
        major_axis*=-1
 
    evecs[0] = to_np(major_axis)
    plot_multi_pca(points_sym.unsqueeze(0), axis = evecs)
    

        
    mcenter_point = sym_vec[0]
    # radial distance from major axis
    projected_point_0 = project_to_axis(vector = mcenter_point, axis = symmetry_plane_normal) 
    dist = torch.norm(projected_point_0) # radial distance from major axis

    projected_point_sym2nonsym = project_to_axis(vector = mcenter_point, axis = sym2nonsym) 
    dist_projected_point_sym2nonsym = torch.norm(projected_point_sym2nonsym) 
    axes0 = [
        
             [to_np(nonsym_major_axis+non_sym_center),to_np(non_sym_center)],
             [to_np(symmetry_plane_normal+pcd_mean),to_np(pcd_mean)],
             [to_np(non_sym_center),to_np(pcd_mean)],
             [to_np(projected_point_0+pcd_mean),to_np(pcd_mean)],
             [to_np(pcd_mean), to_np(mcenter_point+pcd_mean)]
    ]
    plot_axis([anchor_cloud, action_cloud],axes0)
    dotproduct = torch.matmul(mcenter_point,symmetry_plane_normal)/torch.norm(symmetry_plane_normal)
 
 
    sym_cls = torch.sign(torch.matmul(sym_vec, symmetry_plane_normal)) # 1, num_points 
    cts_cls =  torch.matmul(sym_vec,symmetry_plane_normal)/torch.norm(symmetry_plane_normal) 
    # coloring cts_cls for vis (+: red, -: blue)
    print("cts_cls.shape",cts_cls.shape,points_sym.shape)
    color = (cts_cls/torch.abs(torch.max(cts_cls))*255)
    color_cts = torch.zeros(points_sym.shape).to(points_sym.device).double()
 
    color_cts[cts_cls>=0,0] = color[cts_cls>=0]
    color_cts[cts_cls<0,2] = torch.abs(color[cts_cls<0])
    color_cts[cts_cls>=0]
    
    # another color scheme (+: red, -: green, 0: yellow )
    # https://stackoverflow.com/questions/6394304/algorithm-how-do-i-fade-from-red-to-green-via-yellow-using-rgb-values
    color = (cts_cls/torch.abs(torch.max(cts_cls))*(255/2))+(255/2)
    color_cts = torch.zeros(points_sym.shape).to(points_sym.device).double()
 
    color_cts[:,0] = torch.minimum(torch.ones(color.shape).to(color.device)*255,color*2)
    color_cts[:,1] = torch.minimum(torch.ones(color.shape).to(color.device)*255,(255-color)*2)
    plot_color([points_sym,points_nonsym],[color_cts,None])
    return sym_cls.unsqueeze(0), cts_cls.unsqueeze(0)

In [ ]:
def get_sym_label_pca_gripper(action_cloud, anchor_cloud, action_class, anchor_class, object_type = 'bottle'):
    assert 0 in [action_class, anchor_class], "class 0 must be here somewhere as the manipulation object of interest"
    if action_class == 0:
        sym_breaking_class = action_class
        center_class = anchor_class
        points_sym = action_cloud[0]
        points_nonsym = anchor_cloud[0]
    elif anchor_class == 0:
        sym_breaking_class = anchor_class
        center_class = action_class
        points_sym = anchor_cloud[0]
        points_nonsym = action_cloud[0]

    non_sym_center = points_nonsym.mean(axis=0)
    points_sym_np = to_np(points_sym)
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_sym_np)
    pcd_mean, pcd_cov = pcd.compute_mean_and_covariance()
    evals, evecs = np.linalg.eig(pcd_cov) 
    evecs = np.transpose(evecs)
    major_axis = evecs[-1]
    if object_type == 'bottle':
        major_axis = evecs[0]

    points_nonsym_np = to_np(points_nonsym)
    pcd_nonsym = o3d.geometry.PointCloud()
    pcd_nonsym.points = o3d.utility.Vector3dVector(points_nonsym_np)
    pcd_nonsym_mean, pcd_nonsym_cov = pcd_nonsym.compute_mean_and_covariance()
    nonsym_evals, nonsym_evecs = np.linalg.eig(pcd_nonsym_cov) 
    nonsym_evecs = np.transpose(nonsym_evecs)
    nonsym_major_axis = nonsym_evecs[0]
    nonsym_major_axis = to_torch(nonsym_major_axis, device_tensor = non_sym_center).float()
 
    
    plot_multi_pca(points_sym.unsqueeze(0), axis = evecs)
    plot_multi_pca(points_nonsym.unsqueeze(0), axis = nonsym_evecs)
    
    major_axis = to_torch(major_axis, device_tensor = non_sym_center)
    pcd_mean = to_torch(pcd_mean, device_tensor = non_sym_center)  
    projected_point = project_to_axis(vector = non_sym_center-pcd_mean, axis = major_axis) + pcd_mean 
 
    
    air_point = non_sym_center+(pcd_mean-projected_point)
    sym_vec = points_sym - pcd_mean
    sym2nonsym = air_point-pcd_mean
    if object_type == 'bottle':
        sym2nonsym = torch.cross(sym2nonsym, major_axis)
        if torch.matmul(sym2nonsym.float(),nonsym_major_axis)<0:
            sym2nonsym*=-1
 
    vec = to_np(projected_point - pcd_mean)
    axes = [[to_np(pcd_mean), to_np(projected_point)],
            [to_np(projected_point), to_np(non_sym_center)],
            [to_np(pcd_mean), to_np(major_axis+pcd_mean)],
            [to_np(pcd_mean), to_np(air_point)]]
    # plot_axis([anchor_cloud, action_cloud], axes) 
#     print(torch.matmul(sym_vec, sym2nonsym))
    mcenter_point = sym_vec[0]
    # radial distance from major axis
    projected_point_0 = project_to_axis(vector = mcenter_point, axis = major_axis) 
    dist = torch.norm(projected_point_0 - mcenter_point) # radial distance from major axis

    projected_point_sym2nonsym = project_to_axis(vector = mcenter_point, axis = sym2nonsym) 
    dist_projected_point_sym2nonsym = torch.norm(projected_point_sym2nonsym) 
    axes0 = [
             [to_np(non_sym_center), to_np(pcd_mean)],
             [to_np(projected_point_sym2nonsym+pcd_mean), to_np(pcd_mean)],
             [to_np(sym2nonsym+pcd_mean),to_np(pcd_mean)],
             [to_np(mcenter_point+pcd_mean),to_np(pcd_mean)],
             [to_np(major_axis+pcd_mean),to_np(pcd_mean)]]
    plot_axis([anchor_cloud, action_cloud],axes0)
    dotproduct = torch.matmul(mcenter_point,sym2nonsym)/torch.norm(sym2nonsym)
 
    project_to_axis(vector = points_sym[0]-pcd_mean, axis = major_axis)
    sym_cls = torch.sign(torch.matmul(sym_vec, sym2nonsym)) # 1, num_points 
    cts_cls =  torch.matmul(sym_vec,sym2nonsym)/torch.norm(sym2nonsym) 
    cts_cls_norm = cts_cls/torch.max(torch.abs(cts_cls))
    nonsym_vec = points_nonsym - non_sym_center
    print("nonsym_vec.type", nonsym_vec.dtype,  nonsym_major_axis.dtype)
    cts_cls_nonsym = torch.matmul(nonsym_vec,nonsym_major_axis)/torch.norm(nonsym_major_axis) 
 
    # coloring cts_cls for vis (+: red, -: blue)
    color = (cts_cls/torch.abs(torch.max(cts_cls))*255)
    color_cts = torch.zeros(points_sym.shape).to(points_sym.device).double()
 
    color_cts[cts_cls>=0,0] = color[cts_cls>=0]
    color_cts[cts_cls<0,2] = torch.abs(color[cts_cls<0])
    color_cts[cts_cls>=0]
    
    # another color scheme (+: red, -: green, 0: yellow )
    # https://stackoverflow.com/questions/6394304/algorithm-how-do-i-fade-from-red-to-green-via-yellow-using-rgb-values
    color = (cts_cls/torch.abs(torch.max(cts_cls))*(255/2))+(255/2)
    color_cts = torch.zeros(points_sym.shape).to(points_sym.device).double()
 
    color_cts[:,0] = torch.minimum(torch.ones(color.shape).to(color.device)*255,color*2)
    color_cts[:,1] = torch.minimum(torch.ones(color.shape).to(color.device)*255,(255-color)*2)
    
#     plot_color([points_sym,points_nonsym],[color_cts,None])
    
    color_nonsym = (cts_cls_nonsym/torch.abs(torch.max(cts_cls_nonsym))*(255/2))+(255/2)
    color_cts_nonsym = torch.zeros(points_nonsym.shape).to(points_nonsym.device).double()
 
    color_cts_nonsym[:,0] = torch.minimum(torch.ones(color_nonsym.shape).to(color_nonsym.device)*255,color_nonsym*2)
    color_cts_nonsym[:,1] = torch.minimum(torch.ones(color_nonsym.shape).to(color_nonsym.device)*255,(255-color_nonsym)*2)
    
    plot_color([points_sym,points_nonsym],[color_cts,color_cts_nonsym])
    return cts_cls_nonsym.unsqueeze(0), cts_cls.unsqueeze(0)

In [ ]:
def get_sym_label_pca_test(rack, action_cloud, anchor_cloud, action_class, anchor_class, object_type = 'bowl'):
    assert 0 in [action_class, anchor_class], "class 0 must be here somewhere as the manipulation object of interest"
    if action_class == 0:
        sym_breaking_class = action_class
        center_class = anchor_class
        points_sym = action_cloud[0]
        points_nonsym = anchor_cloud[0]
    elif anchor_class == 0:
        sym_breaking_class = anchor_class
        center_class = action_class
        points_sym = anchor_cloud[0]
        points_nonsym = action_cloud[0]
    points_ref = rack[0]

    non_sym_center = points_nonsym.mean(axis=0)
    points_sym_np = to_np(points_sym)
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_sym_np)
    pcd_mean, pcd_cov = pcd.compute_mean_and_covariance()
    evals, evecs = np.linalg.eig(pcd_cov) 
    evecs = np.transpose(evecs)
    major_axis = evecs[0]
    
    plot_multi_pca(points_sym.unsqueeze(0), axis = evecs,index_list = [0,1,2])   
    
    points_ref_center = points_ref.mean(axis=0)
    points_ref_np = to_np(points_ref)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_ref_np)
    rack_pcd_mean, pcd_cov = pcd.compute_mean_and_covariance()
    evals, rack_evecs = np.linalg.eig(pcd_cov) 
    rack_evecs = np.transpose(rack_evecs)
    rack_major_axis = rack_evecs[0]
 
    major_axis = to_torch(major_axis, device_tensor = non_sym_center)
    rack_major_axis = to_torch(rack_major_axis,device_tensor = non_sym_center)
    pcd_mean = to_torch(pcd_mean, device_tensor = non_sym_center)  
    projected_point = project_to_axis(vector = non_sym_center-pcd_mean, axis = major_axis) + pcd_mean 
  
    sym_vec = points_sym - pcd_mean
 
    sym2nonsym = torch.cross(rack_major_axis,major_axis)
    sym2nonsym = rack_major_axis
    
    axes = [[to_np(pcd_mean), to_np(sym2nonsym+pcd_mean)],
            [to_np(pcd_mean), to_np(major_axis+pcd_mean)],
#             [to_np(pcd_mean), to_np(major_axis+pcd_mean)],
#             [to_np(pcd_mean), to_np(air_point)]
           ]
    plot_axis([anchor_cloud, action_cloud], axes) 
    
    sym_cls = torch.sign(torch.matmul(sym_vec, sym2nonsym)) # 1, num_points 
    cts_cls =  torch.matmul(sym_vec,sym2nonsym)/torch.norm(sym2nonsym) 
    cts_cls_norm = cts_cls/torch.max(torch.abs(cts_cls))
 
 
    # coloring cts_cls for vis (+: red, -: blue)
    color = (cts_cls/torch.abs(torch.max(cts_cls))*255)
    color_cts = torch.zeros(points_sym.shape).to(points_sym.device).double()
 
    color_cts[cts_cls>=0,0] = color[cts_cls>=0]
    color_cts[cts_cls<0,2] = torch.abs(color[cts_cls<0])
    color_cts[cts_cls>=0]
    
    # another color scheme (+: red, -: green, 0: yellow )
    # https://stackoverflow.com/questions/6394304/algorithm-how-do-i-fade-from-red-to-green-via-yellow-using-rgb-values
    color = (cts_cls/torch.abs(torch.max(cts_cls))*(255/2))+(255/2)
    color_cts = torch.zeros(points_sym.shape).to(points_sym.device).double()
 
    color_cts[:,0] = torch.minimum(torch.ones(color.shape).to(color.device)*255,color*2)
    color_cts[:,1] = torch.minimum(torch.ones(color.shape).to(color.device)*255,(255-color)*2)
 
    plot_color([points_sym, points_nonsym, points_ref],[color_cts,None, None])
    return cts_cls_nonsym.unsqueeze(0), cts_cls.unsqueeze(0)

In [ ]:
from equivariant_pose_graph.utils.symmetry_utils import get_sym_label_pca_test_bottle_graspable
num_classes = 3
data_idx = 1
cloud_types = ['init', 'pre_grasp', 'post_grasp', 'teleport', 'post_place', 'final']
cloud_type = 'plan2'
timestamp = '2022-07-13_093105/2022-07-13-09-31-05'
data_path = Path(f'/home/exx/media/DataDrive/singularity_chuerp/epg_results/residual_flow_test/residual_flow_test_partial_cloud_initpose/{timestamp}/renders/')
# data_path = Path('/home/exx/Documents/ndf_robot/bottle_train_data_ndf_cons_3/renders') 
action_class, anchor_class = 2,0
# data_path = Path('/home/exx/Documents/ndf_robot/bowl_test_new_3_pregrasp/renders') 
# action_class, anchor_class = 2,0

# data_path = Path('/home/exx/Documents/ndf_robot/bowl_train_data_ndf_cons_3/renders') 
# action_class, anchor_class = 0,1
point_data = np.load(data_path / f'{data_idx}_{cloud_type}_obj_points.npz', allow_pickle = True)

points_action, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= action_class, anchor_class= anchor_class)
points_rack, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= 1, anchor_class= anchor_class)
ans_dict = get_sym_label_pca_test_bottle_graspable(points_rack, 
                                                   action_cloud = points_action, 
                                                   anchor_cloud = points_anchor, 
                                                   action_class= action_class, 
                                                   anchor_class= anchor_class, 
                                                   normalize_dist= True,
                                                   object_type = 'bowl')
plot_multi([points_action, points_anchor, points_rack])
#                [ans_dict['cts_cls'][0], ans_dict['cts_cls_nonsym'][0], None])
print("cts_cls.shape", ans_dict['cts_cls'].shape)
plot_color([points_anchor[0], points_action[0], points_rack[0]],
               [ans_dict['cts_cls'][0], ans_dict['cts_cls_nonsym'][0], None])

In [ ]:
from equivariant_pose_graph.utils.symmetry_utils import get_sym_label_pca_test_bottle_graspable
num_classes = 3
data_idx = 4
cloud_types = ['init', 'pre_grasp', 'post_grasp', 'teleport', 'post_place', 'final']
cloud_type = 'init'
timestamp = '2022-07-12_142311/2022-07-12-14-23-11'
data_path = Path(f'/home/exx/media/DataDrive/singularity_chuerp/epg_results/residual_flow_test/residual_flow_test_partial_cloud_initpose/{timestamp}/renders/')
# data_path = Path('/home/exx/Documents/ndf_robot/bottle_train_data_ndf_cons_3/renders') 
action_class, anchor_class = 2,0
# data_path = Path('/home/exx/Documents/ndf_robot/bowl_test_new_3_pregrasp/renders') 
# action_class, anchor_class = 2,0

# data_path = Path('/home/exx/Documents/ndf_robot/bowl_train_data_ndf_cons_3/renders') 
# action_class, anchor_class = 0,1
point_data = np.load(data_path / f'{data_idx}_{cloud_type}_obj_points.npz', allow_pickle = True)

points_action, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= action_class, anchor_class= anchor_class)
points_rack, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= 1, anchor_class= anchor_class)
ans_dict = get_sym_label_pca_test_bottle_graspable(points_rack, 
                                                   action_cloud = points_action, 
                                                   anchor_cloud = points_anchor, 
                                                   action_class= action_class, 
                                                   anchor_class= anchor_class, 
                                                   normalize_dist= True,
                                                   object_type = 'bowl')
print("cts_cls.shape", ans_dict['cts_cls'].shape)
plot_color([points_anchor[0], points_action[0], points_rack[0]],
               [ans_dict['cts_cls'][0], ans_dict['cts_cls_nonsym'][0], None])

In [ ]:
from equivariant_pose_graph.utils.symmetry_utils import get_sym_label_pca_test_bottle_graspable
num_classes = 3
data_idx = 1
cloud_types = ['init', 'pre_grasp', 'post_grasp', 'teleport', 'post_place', 'final']
cloud_type = 'pre_grasp'

data_path = Path('/home/exx/Documents/ndf_robot/bottle_train_data_ndf_cons_3/renders') 
action_class, anchor_class = 2,0
# data_path = Path('/home/exx/Documents/ndf_robot/bowl_test_new_3_pregrasp/renders') 
# action_class, anchor_class = 2,0

# data_path = Path('/home/exx/Documents/ndf_robot/bowl_train_data_ndf_cons_3/renders') 
# action_class, anchor_class = 0,1
point_data = np.load(data_path / f'{data_idx}_{cloud_type}_obj_points.npz', allow_pickle = True)

points_action, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= action_class, anchor_class= anchor_class)
points_rack, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= 1, anchor_class= anchor_class)
#v cts_cls_nonsym, cts_cls = get_sym_label_pca_test(action_cloud = points_action, anchor_cloud = points_anchor, action_class= action_class, anchor_class= anchor_class, object_type = 'bowl') # num_points, 1
ans_dict = get_sym_label_pca_test_bottle_graspable(points_rack, action_cloud = points_action, anchor_cloud = points_anchor, action_class= action_class, anchor_class= anchor_class, object_type = 'bowl')

In [ ]:
data_idx = 5
cloud_types = ['init', 'pre_grasp', 'post_grasp', 'teleport', 'post_place', 'final']
cloud_type = 'pre_grasp'
timestamp = '2022-07-12_142311/2022-07-12-14-23-11'
data_path = Path(f'/home/exx/media/DataDrive/singularity_chuerp/epg_results/residual_flow_test/residual_flow_test_partial_cloud_initpose/{timestamp}/renders/')
point_data = np.load(data_path / f'{data_idx}_{cloud_type}_obj_points.npz', allow_pickle = True)
points_action, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= 2, anchor_class= 0)
points_rack, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= 1, anchor_class= 0)
plot_multi([points_action,points_anchor, points_rack])

In [ ]:
 
data_idx = 1
cloud_types = ['init', 'pre_grasp', 'post_grasp', 'teleport', 'post_place', 'final']
cloud_type = 'pre_grasp'
data_path = Path('/home/exx/Documents/ndf_robot/bottle_train_data_ndf_cons_3/renders')
point_data = np.load(data_path / f'{data_idx}_{cloud_type}_obj_points.npz', allow_pickle = True)
points_action, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= 2, anchor_class= 0)
points_rack, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= 1, anchor_class= 0)
plot_multi([points_action,points_anchor, points_rack])

In [ ]:
data_idx = 5
cloud_types = ['init', 'pre_grasp', 'post_grasp', 'teleport', 'post_place', 'final']
cloud_type = 'pre_grasp'
timestamp = '2022-07-12_142311/2022-07-12-14-23-11'
data_path = Path(f'/home/exx/media/DataDrive/singularity_chuerp/epg_results/residual_flow_test/residual_flow_test_partial_cloud_initpose/{timestamp}/renders/')
point_data = np.load(data_path / f'{data_idx}_{cloud_type}_obj_points.npz', allow_pickle = True)
points_action, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= 2, anchor_class= 0)
points_rack, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= 1, anchor_class= 0)
plot_multi([points_action,points_anchor, points_rack])

data_idx = 5
cloud_types = ['init', 'pre_grasp', 'post_grasp', 'teleport', 'post_place', 'final']
cloud_type = 'pre_grasp'
timestamp = '2022-07-12_142311/2022-07-12-14-23-11'
data_path = Path(f'/home/exx/media/DataDrive/singularity_chuerp/epg_results/residual_flow_test/residual_flow_test_partial_cloud_initpose/{timestamp}/renders/')
point_data = np.load(data_path / f'{data_idx}_{cloud_type}_obj_points.npz', allow_pickle = True)
points_action, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= 2, anchor_class= 0)
points_action, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= , anchor_class= 0)
plot_multi([points_action,points_anchor])

In [ ]:
cloud_type = 'post_grasp'
data_path = Path('/home/exx/media/DataDrive/singularity_chuerp/epg_results/residual_flow_test/residual_flow_test_partial_cloud_initpose/2022-07-12_134501/2022-07-12-13-45-01/renders/')
point_data = np.load(data_path / f'{data_idx}_{cloud_type}_obj_points.npz', allow_pickle = True)
points_action, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= 2, anchor_class= 0)
plot_multi([points_action,points_anchor])

In [ ]:
num_classes = 3
data_idx = 0
cloud_types = ['init', 'pre_grasp', 'post_grasp', 'teleport', 'post_place', 'final']
cloud_type = 'init'
data_path = Path('/home/exx/Documents/ndf_robot/bottle_train_new_3_pregrasp/renders') 
point_data = np.load(data_path / f'{data_idx}_{cloud_type}_obj_points.npz', allow_pickle = True)

points_action, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= 2, anchor_class= 0)
ans_dict = model.get_transform(
            points_action, points_anchor, action_class=2, anchor_class=0)
pred_T_action = ans_dict['pred_T_action']
points_action_pred = pred_T_action.transform_points(points_action)
 

plot_multi([points_action,points_anchor,points_action_pred])
print(points_action_pred.shape)

gripper_pred_center = points_action_pred[0].mean(axis=0) # 1,3
anchor = points_anchor[0]
anchor_center = anchor.mean(axis=0)
points_sym_np = to_np(anchor)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_sym_np)
pcd_mean, pcd_cov = pcd.compute_mean_and_covariance()
evals, evecs = np.linalg.eig(pcd_cov) 
evecs = np.transpose(evecs)
major_axis = evecs[0]
major_axis = to_torch(major_axis, device_tensor= gripper_pred_center)
plot_multi_pca(points_anchor, axis = evecs)

bottle2gripper_center = gripper_pred_center - anchor_center
projected_point_on_anchor_axis = torch.matmul(bottle2gripper_center.float(), major_axis.float())/torch.norm(major_axis.float())*major_axis+anchor_center.float()
radial_vector_towards_gripper = gripper_pred_center-projected_point_on_anchor_axis
points_anchor-=radial_vector_towards_gripper*1
axes0 = [
         [to_np(gripper_pred_center), to_np(projected_point_on_anchor_axis)],
         [to_np(major_axis+anchor_center), to_np(anchor_center)],
         [to_np(gripper_pred_center),to_np(anchor_center)],
     ]
plot_axis([points_action, points_anchor, points_action_pred],axes0)

In [ ]:
num_classes = 3
data_idx = 1
cloud_types = ['init', 'pre_grasp', 'post_grasp', 'teleport', 'post_place', 'final']
cloud_type = 'init'
data_path = Path('/home/exx/Documents/ndf_robot/bottle_train_new_3_pregrasp/renders') 
point_data = np.load(data_path / f'{data_idx}_{cloud_type}_obj_points.npz', allow_pickle = True)

points_action, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= 2, anchor_class= 0)
sym_cls, cts_cls = get_sym_label_pca_gripper(action_cloud = points_action, anchor_cloud = points_anchor, \
                                                action_class= 2, anchor_class= 0) # num_points, 1
print((sym_cls>0).shape)

points_anchor_left = points_anchor[sym_cls>0]
points_anchor_right = points_anchor[sym_cls<=0]

plot_multi([points_anchor_left,points_anchor_right,points_action])

In [ ]:
num_classes = 3
data_idx = 1
cloud_types = ['init', 'pre_grasp', 'post_grasp', 'teleport', 'post_place', 'final']
cloud_type = 'pre_grasp'
data_path = Path('/home/exx/Documents/ndf_robot/src/ndf_robot/test_bowl_arbitrary') 
# data_path = Path('/home/exx/Documents/ndf_robot/bowl_train_data_ndf_cons_3/renders') 
point_data = np.load(data_path / f'{data_idx}_{cloud_type}_obj_points.npz', allow_pickle = True)

points_action, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= 0, anchor_class= 1)
sym_cls, cts_cls = get_sym_label_pca_bowl_place(action_cloud = points_action, anchor_cloud = points_anchor, \
                                                action_class= 0, anchor_class= 1) # num_points, 1
print((sym_cls>0).shape)

points_action_left = points_action[sym_cls>0]
points_action_right = points_action[sym_cls<=0]

plot_multi([points_action_left,points_action_right,points_anchor])

In [ ]:
num_classes = 3
data_idx = 1
cloud_types = ['init', 'pre_grasp', 'post_grasp', 'teleport', 'post_place', 'final']
cloud_type = 'pre_grasp'
data_path = Path('/home/exx/Documents/ndf_robot/bottle_train_data_ndf_cons_3/renders') 
# data_path = Path('/home/exx/Documents/ndf_robot/bowl_train_data_ndf_cons_3/renders') 
point_data = np.load(data_path / f'{data_idx}_{cloud_type}_obj_points.npz', allow_pickle = True)

points_action, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= 0, anchor_class= 1)
sym_cls, cts_cls = get_sym_label_pca_bowl_place(action_cloud = points_action, anchor_cloud = points_anchor, \
                                                action_class= 0, anchor_class= 1, object_type = 'bottle') # num_points, 1
print((sym_cls>0).shape)

points_action_left = points_action[sym_cls>0]
points_action_right = points_action[sym_cls<=0]

plot_multi([points_action_left,points_action_right,points_anchor])

In [ ]:
num_classes = 3
data_idx = 2
cloud_types = ['init', 'pre_grasp', 'post_grasp', 'teleport', 'post_place', 'final']
cloud_type = 'teleport'
# data_path = Path('/home/exx/Documents/ndf_robot/src/ndf_robot/test_bottle_arbitrary_Seed2') 
data_path = Path('/home/exx/Documents/ndf_robot/bottle_train_data_ndf_cons_3/renders') 
point_data = np.load(data_path / f'{data_idx}_{cloud_type}_obj_points.npz', allow_pickle = True)

points_action, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= 0, anchor_class= 1)
sym_cls, cts_cls = get_sym_label_pca_bowl_place(action_cloud = points_action, anchor_cloud = points_anchor, \
                                                action_class= 0, anchor_class= 1) # num_points, 1
print((sym_cls>0).shape)

points_action_left = points_action[sym_cls>0]
points_action_right = points_action[sym_cls<=0]

plot_multi([points_action_left,points_action_right,points_anchor])

In [ ]:
num_classes = 3
data_idx = 2
cloud_types = ['init', 'pre_grasp', 'post_grasp', 'teleport', 'post_place', 'final']
cloud_type = 'pre_grasp'
data_path = Path('/home/exx/Documents/ndf_robot/src/ndf_robot/test_bottle_arbitrary_Seed2') 
point_data = np.load(data_path / f'{data_idx}_{cloud_type}_obj_points.npz', allow_pickle = True)

points_action, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= 2, anchor_class= 0)
sym_cls = get_sym_label_pca_bottle_playground(action_cloud = points_action, anchor_cloud = points_anchor, action_class= 2, anchor_class= 0) # num_points, 1
print((sym_cls>0).shape)

points_anchor_left = points_anchor[sym_cls>0]
points_anchor_right = points_anchor[sym_cls<=0]

plot_multi([points_anchor_left,points_anchor_right,points_action])

In [ ]:
import open3d as o3d
import numpy as np
num_classes = 3
data_idx = 2
cloud_types = ['init', 'pre_grasp', 'post_grasp', 'teleport', 'post_place', 'final']
cloud_type = 'pre_grasp'
data_path = Path('/home/exx/Documents/ndf_robot/bowl_test_new_3_pregrasp/renders') 
# data_path = Path('/home/exx/Documents/ndf_robot/bowl_train_data_ndf_cons_3/renders') 
# data_path = Path('/home/exx/Documents/ndf_robot/src/ndf_robot/test_bowl_upright') 
point_data = np.load(data_path / f'{data_idx}_{cloud_type}_obj_points.npz', allow_pickle = True)
points_action_init, points_anchor = load_data(num_points=1000, point_data=point_data, action_class= 2, anchor_class= 0)
points_anchor_np = points_anchor[0].detach().cpu().numpy()
points_action_np = points_action[0].detach().cpu().numpy()
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_action_np)
pcd_mean, pcd_cov = pcd.compute_mean_and_covariance()
evals, evecs = np.linalg.eig(pcd_cov) 
evecs = np.transpose(evecs)

In [ ]:
plot_multi_pca(points_action, axis = evecs)